# Timeline Analysis Demo

In trace analysis, a timeline refers to a visual representation of a sequence of events occurring over a specific period of time. 
The timeline can help analysts identify patterns, trends, and anomalies, as well as understand dependencies and interactions between different components in the system. 

In HTA, creating a timeline plot involves the following steps:

1. Load, parse, or transform the traces into a single DataFrame. This DataFrame may contain traces for a single rank, multiple ranks, or multiple traces. The DataFrame can contain either CPU events, GPU events, or both.
2. Create a new DataFrame for the timeline plot by calling `prepare_timeline_events()`. This DataFrame will contain the calibrated starting and ending timestamps, the task labels, as well as other values for display.
3. Plot the timeline by calling `plot_events_timeline()`. The timeline figure can be saved to an HTML file or output as an HTML string representing the figure.

To customize the plot, you can create a `TimelinePlotSetting` object with your desired customizations and pass it to `prepare_timeline_events()` and `plot_events_timeline()`.

To streamline this process, HTA also provides a Timeline class to combine step 2 and 3.

The following cell illustrates creating several types of timeline plots following the above steps. For a demonstration purpose, we load parsed traces from several .csv files to reduce the trace parsing time.

## Data-driven timeline plot

The timeline plotting utilities offered by HTA are inherently data-driven. These tools can be used to visualize a wide range of scenarios, including individual traces, traces across multiple ranks, multiple job traces, and specific filtered events. The flexibility of these utilities lies in their ability to adapt to different contexts; the only requirements are modifying the input DataFrame and adjusting the filter function as needed.

In [ ]:
# ON_GITHUB = True
ON_GITHUB = False
if ON_GITHUB:
    import plotly.io as pio
    pio.renderers.default = "svg"

## Load the traces

In [ ]:
from hta.common.trace import Trace
from hta.common.trace_call_graph import CallGraph
import hta
import os
from pathlib import Path

# Load the traces in a folder
base_data_dir = str(Path(hta.__file__).parent.parent.joinpath("tests/data"))
trace_dir: str = os.path.join(base_data_dir, "trace_filter")
t = Trace(trace_dir=os.path.join(base_data_dir, "trace_filter"))
t.parse_traces()

# Decode the symbol columns (i.e., `name` and `cat`) 
t.decode_symbol_ids()

# Construct the call stacks for two ranks
cg = CallGraph(t, ranks=[0, 1])

# Save a dataframe and symbol table for subsequent references
df = t.get_trace(0)
symbol_table = t.symbol_table

## Plot the timeline using the Timeline class

The Timeline class is a simply way to create a timeline plot. By default, a Timeline object generates all timeline events ad then plots them in a single plot.

In [ ]:
from hta.common.timeline import Timeline, PlotFormat

Timeline(df, symbol_table).plot("a simple timeline")

## Tailored Timeline with Trace Filters

You can achive better targeted timeline analysis by applying a tailored filter object to the input trace data frame, ensuring a focused and relevant analysis of the specific events of interest.

In [ ]:
from hta.common.trace_filter import GPUKernelFilter

Timeline(df, symbol_table, filter_func=GPUKernelFilter()).plot("a timeline of GPU events")

## Custom Trace Filtering for Timeline Plotting

To achieve finer control over your analysis, you can create a custom Trace Filter function. This allows you to selectively target the specific trace events in your timeline analysis.

In [ ]:
import pandas as pd
from hta.common.trace import TraceSymbolTable
from hta.common.trace_filter import Filter, IterationIndexFilter, NameFilter, GPUKernelFilter

class MyKernelFilter(Filter):
    def __init__(self, name_pattern: str, iteration_index: int) -> None:
        self.name_pattern = name_pattern
        self.iteration_index = iteration_index
        
    def __call__(self, df: pd.DataFrame, symbol_table: TraceSymbolTable) -> pd.DataFrame:
        df = IterationIndexFilter(self.iteration_index)(df, symbol_table)
        df = GPUKernelFilter()(df, symbol_table)
        df = NameFilter(self.name_pattern)(df, symbol_table)
        return df

my_filter = MyKernelFilter(r"(.+embedding)|(^nccl)|(Memcpy)", iteration_index=1)
Timeline(df, symbol_table, filter_func=my_filter).plot("a simple timeline")

## Align GPU events with corresponding CPU operators or annotations

To provid a coherent and integrated view of the activity across both CPU and GPU devices, you can align the GPU events with their respective CPU operators or annotations by shifting the starting time of the CPU events to their first GPU kernels's starting time.

In [ ]:
from hta.common.timeline import align_module_with_kernels

df_aligned = align_module_with_kernels(df, module_list=["## forward ##", "## backward ##", "## optimizer ##"], sym_table=symbol_table)
Timeline(df_aligned, symbol_table).plot("operator highlight")